In [ ]:
# %% 

import pandas as pd 
import numpy as np 
import xgboost as xgb  # Importando o XGBoost
from sklearn.pipeline import Pipeline
from pre_processing import importar_dados, preprocessar_dados
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score
from sklearn.metrics import mean_absolute_error
from scipy.stats import randint
import joblib

In [ ]:
# %% 

# Utilizando as funções criadas para importar e pré-processar os dados
X_train, X_val, X_test, y_train, y_val, y_test = importar_dados() 
preprocessor = preprocessar_dados()
X_train.head()

,ANO_VENCIMENTO,MES_VENCIMENTO,TRIMESTRE,VALOR_FATURA_lag1,VALOR_FATURA_lag2
0,2023,8,3,115.00,448.35
1,2023,4,2,8024.27,4845.25
2,2022,4,2,55.20,55.20
3,2024,3,1,69.89,68.00
4,2023,6,2,5430.86,53.04


In [ ]:
# %%

# Criando o modelo de regressão XGBoost
# O 'random_state=42' garante que os resultados sejam reprodutíveis.
model = xgb.XGBRegressor(random_state=42)

In [ ]:
# %% 

# Pipeline
# O pipeline permite integrar o pré-processamento e o modelo em um único fluxo de trabalho.
# Isso facilita a execução do pré-processamento e a aplicação do modelo de forma sequencial e correta.
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Primeiro, aplica o pré-processamento aos dados.
    ('regressor', model)  # Depois, treina o modelo XGBoost.
])

In [ ]:
# %% 

# Definindo os parâmetros a serem testados durante a busca de hiperparâmetros.
# Estamos utilizando uma distribuição aleatória para parâmetros como 'n_estimators' e 'max_depth'.
param_distributions = {
    'regressor__colsample_bytree': [0.3, 0.5, 0.7],
    'regressor__max_depth': [3, 5, 7],
    'regressor__learning_rate': [0.01, 0.1, 0.2],
    'regressor__n_estimators': [50, 100, 200]
}

# RandomizedSearchCV realiza uma busca aleatória nos parâmetros definidos acima.
# Ele testa combinações aleatórias e avalia qual combinação tem o melhor desempenho.
random_search = RandomizedSearchCV(
    pipeline,  # O pipeline com o pré-processamento e o modelo.
    param_distributions,  # O espaço de busca para os hiperparâmetros.
    n_iter=10,  # Número de iterações para testar combinações aleatórias.
    cv=3,  # Número de divisões (folds) para validação cruzada durante a busca.
    scoring='neg_mean_absolute_error',  # A métrica a ser otimizada (erro absoluto médio negativo).
    n_jobs=-1,  # Utiliza todos os núcleos de processamento disponíveis para acelerar a busca.
    random_state=42  # Garante a reprodutibilidade da busca aleatória.
)

# Realiza a busca de hiperparâmetros ajustando o modelo aos dados de treinamento.
random_search.fit(X_train, y_train)
# Exibe os melhores parâmetros encontrados durante a busca.
print("Melhores hiperparâmetros:", random_search.best_params_)

Melhores hiperparâmetros: {'regressor__n_estimators': 100, 'regressor__max_depth': 3, 'regressor__learning_rate': 0.1, 'regressor__colsample_bytree': 0.3}


In [ ]:
# %% 

# Após a busca de hiperparâmetros, obtemos o modelo final treinado com os melhores parâmetros.
xgboost_model = random_search.best_estimator_

# %% [markdown]

 ### Avaliando Resultado dos modelos

In [ ]:
#%%

# Avalia o modelo com a validação cruzada
validacao_cruzada = KFold(n_splits=10, shuffle=True, random_state=42)
cross_val_scores = cross_val_score(model, X_val, y_val, cv=validacao_cruzada)
acuracia_media_xgb = cross_val_scores.mean()
print(cross_val_scores)
print("Acurácia média do XGBoost:", acuracia_media_xgb)

# Faz previsões com o pipeline ajustado
y_pred = xgboost_model.predict(X_val)

[ 0.61035049  0.62388557  0.30463785  0.51918304  0.27939767  0.63593292
  0.46004999 -0.18117762  0.62432539  0.72670364]
Acurácia média do XGBoost: 0.4603288948535919


# %% [markdown]

 1. Mean Absolute Error (MAE)
 O MAE calcula o erro médio absoluto entre as previsões e os valores reais. Ele é útil para entender o erro médio sem considerar a direção (positivo ou negativo).

In [ ]:
#%%

from sklearn.metrics import mean_absolute_error

# y_val: valores reais, y_pred: valores previstos
mae = mean_absolute_error(y_val, y_pred)

# Exibindo o valor do MAE
print(f"Mean Absolute Error (MAE): {mae}")

Mean Absolute Error (MAE): 856.4199723378036


# %% [markdown]

 2. Mean Squared Error (MSE)
 O MSE calcula o erro quadrado médio. Ele penaliza mais os erros grandes, pois a diferença entre o valor real e o previsto é elevada ao quadrado

In [ ]:
#%%

from sklearn.metrics import mean_squared_error

# y_val: valores reais, y_pred: valores previstos
mse = mean_squared_error(y_val, y_pred)

# Exibindo o valor do MSE
print(f"Mean Squared Error (MSE): {mse}")

Mean Squared Error (MSE): 13493929.087162755


# %% [markdown]

 3. Root Mean Squared Error (RMSE)
 O RMSE é a raiz quadrada do MSE. Ele traz a medida do erro para a mesma escala dos dados originais, o que torna mais fácil de interpretar.

In [ ]:
#%%

import numpy as np
# Calculando o RMSE como a raiz quadrada do MSE
rmse = np.sqrt(mse)

# Exibindo o valor do RMSE
print(f"Root Mean Squared Error (RMSE): {rmse}")

Root Mean Squared Error (RMSE): 3673.4083746791284


# %% [markdown]

 4. Coefficient of Determination (R²)
 O R² (ou coeficiente de determinação) mede a proporção da variação nos dados que o modelo é capaz de explicar. Um valor de 1 significa explicação perfeita, enquanto um valor de 0 significa que o modelo não explicou nada além da média.

In [ ]:
#%%

from sklearn.metrics import r2_score

# y_val: valores reais, y_pred: valores previstos
r2 = r2_score(y_val, y_pred)

# Exibindo o valor do R²
print(f"Coefficient of Determination (R²): {r2}")

Coefficient of Determination (R²): 0.6852733492851257


# %% [markdown]

 5. Explained Variance Score
 A Expained Variance Score calcula a variação explicada pelo modelo. Um valor próximo de 1 significa que o modelo explicou a maior parte da variação dos dados.

In [ ]:
#%%

from sklearn.metrics import explained_variance_score

# y_val: valores reais, y_pred: valores previstos
explained_variance = explained_variance_score(y_val, y_pred)

# Exibindo o valor do Explained Variance Score
print(f"Explained Variance Score: {explained_variance}")

Explained Variance Score: 0.6853822643454331


In [ ]:
# %% 

# Após treinar o modelo e validá-lo, podemos salvar o modelo final em um arquivo para uso posterior.
# O arquivo será salvo com o nome 'xgboost_model.joblib' para fácil carregamento e uso no futuro.
joblib.dump(xgboost_model, 'xgboost_model.joblib')

['xgboost_model.joblib']